In [1]:
import pandas as pd
import numpy as np
import datetime
import importlib
import pickle

import data_utils
import model_utils
import train_utils
import evaluation
import submission
importlib.reload(data_utils)
importlib.reload(model_utils)
importlib.reload(train_utils)
importlib.reload(evaluation)
importlib.reload(submission)

import torch
import torch.nn as nn
from torchtext.data import TabularDataset, Field, RawField, BucketIterator, Iterator

from bert_embedding import BertEmbedding

I0205 13:02:32.531080 140685029754688 file_utils.py:38] PyTorch version 1.4.0 available.


In [2]:
DEVICE = "cuda"

### Training 

In [3]:
train_data, val_data, test_data = data_utils.prep_all_data(
    train_file="train_real.csv", val_file="val.csv", test_file="test.csv", batch_size=32)

In [4]:
importlib.reload(model_utils)
m = model_utils.GRU(num_layer=1)
result = train_utils.train(train_data, val_data, m, device=DEVICE,
                          lr=1e-2, print_freq=5, patience=20, max_epoch=100)
torch.save(result["trained_model"], "./data/model_checkpoints/GRU_Feb01_1layer.mdl")

use fasttext pretained embedding
Epoch: 0, LR: 0.01, Train Loss: 160.3230, Val Loss: 124.9271, Val f1 0.240
Epoch: 5, LR: 0.01, Train Loss: 13.5899, Val Loss: 63.5656, Val f1 0.757
Epoch: 10, LR: 0.01, Train Loss: 1.8686, Val Loss: 69.9664, Val f1 0.756
Epoch: 15, LR: 0.01, Train Loss: 0.3052, Val Loss: 74.8807, Val f1 0.760
Epoch: 20, LR: 0.01, Train Loss: 0.2089, Val Loss: 77.8932, Val f1 0.765
Epoch: 25, LR: 0.001, Train Loss: 0.0958, Val Loss: 80.2702, Val f1 0.764
Epoch: 30, LR: 0.001, Train Loss: 0.0878, Val Loss: 80.8391, Val f1 0.763
Epoch: 35, LR: 0.001, Train Loss: 0.0824, Val Loss: 81.4625, Val f1 0.760
Epoch: 40, LR: 0.001, Train Loss: 0.0748, Val Loss: 81.9883, Val f1 0.762


In [5]:
importlib.reload(model_utils)
m = model_utils.GRU(num_layer=2)
result = train_utils.train(train_data, val_data, m, device=DEVICE,
                          lr=1e-2, print_freq=5, patience=20, max_epoch=100)
torch.save(result["trained_model"], "./data/model_checkpoints/GRU_Feb01_1layer.mdl")

use fasttext pretained embedding
Epoch: 0, LR: 0.01, Train Loss: 169.5542, Val Loss: 137.4552, Val f1 0.144
Epoch: 5, LR: 0.01, Train Loss: 89.4078, Val Loss: 111.3874, Val f1 0.365
Epoch: 10, LR: 0.01, Train Loss: 47.5972, Val Loss: 90.7164, Val f1 0.573
Epoch: 15, LR: 0.01, Train Loss: 33.1999, Val Loss: 91.2367, Val f1 0.628
Epoch: 20, LR: 0.01, Train Loss: 33.9856, Val Loss: 87.6824, Val f1 0.642
Epoch: 25, LR: 0.01, Train Loss: 20.5181, Val Loss: 90.8966, Val f1 0.682
Epoch: 30, LR: 0.001, Train Loss: 11.3304, Val Loss: 93.0876, Val f1 0.703
Epoch: 35, LR: 0.001, Train Loss: 3.1241, Val Loss: 90.4833, Val f1 0.717
Epoch: 40, LR: 0.001, Train Loss: 2.1230, Val Loss: 89.0167, Val f1 0.714
Epoch: 45, LR: 0.001, Train Loss: 1.6321, Val Loss: 88.5537, Val f1 0.712
Epoch: 50, LR: 0.0001, Train Loss: 1.2929, Val Loss: 88.8563, Val f1 0.709
Epoch: 55, LR: 0.0001, Train Loss: 1.1530, Val Loss: 88.9259, Val f1 0.706


In [377]:
train_data_phrase, val_data_phrase, test_data_phrase = data_utils.prep_all_data(
    path="./data/movie_name_condensed_data/", 
    train_file="train_real.csv", val_file="val.csv", test_file="test.csv")

In [305]:
m = model_utils.BaseModelNGram(path="./data/movie_name_condensed_data/")
result = train_utils.train(train_data_phrase, val_data_phrase, m, device=DEVICE,
                          lr=1e-2, print_freq=5, max_epoch=100)
torch.save(result["trained_model"], "./data/model_checkpoints/ngram_phrase_MLP_Jan31.mdl")

Epoch: 0, LR: 0.01, Train Loss: 240.0455, Val Loss: 113.5318, Val f1 0.671
Epoch: 5, LR: 0.01, Train Loss: 3.2005, Val Loss: 95.2006, Val f1 0.782
Epoch: 10, LR: 0.01, Train Loss: 1.0368, Val Loss: 116.4597, Val f1 0.778


In [387]:
m = model_utils.GRU(path="./data/movie_name_condensed_data/")
result = train_utils.train(train_data_phrase, val_data_phrase, m, device=DEVICE,
                          lr=1e-2, print_freq=5, max_epoch=100)
torch.save(result["trained_model"], "./data/model_checkpoints/GRU_phrase_Jan31.mdl")

Epoch: 0, LR: 0.01, Train Loss: 360.3418, Val Loss: 269.7473, Val f1 0.126
Epoch: 5, LR: 0.01, Train Loss: 30.0495, Val Loss: 122.1983, Val f1 0.737
Epoch: 10, LR: 0.01, Train Loss: 3.8849, Val Loss: 136.4315, Val f1 0.764
Epoch: 15, LR: 0.001, Train Loss: 1.1350, Val Loss: 143.5704, Val f1 0.754
Epoch: 20, LR: 0.001, Train Loss: 0.9925, Val Loss: 144.8627, Val f1 0.755


### Ensemble

In [392]:
train_data, val_data, test_data = data_utils.prep_all_data(
    train_file="train_real.csv", val_file="val.csv", test_file="test.csv")
train_data_phrase, val_data_phrase, test_data_phrase = data_utils.prep_all_data(
    path="./data/movie_name_condensed_data/", 
    train_file="train_real.csv", val_file="val.csv", test_file="test.csv")

m_gram = torch.load("./data/model_checkpoints/ngram_MLP_Jan31.mdl")
m_gram_phrase = torch.load("./data/model_checkpoints/ngram_phrase_MLP_Jan31.mdl")
m_GRU = torch.load("./data/model_checkpoints/GRU_Jan31.mdl")
m_GRU_phrase = torch.load("./data/model_checkpoints/GRU_phrase_Jan31.mdl")

/home/molly/anaconda3/lib/python3.6/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'model_utils.GRU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [393]:
importlib.reload(submission)

ensemble = submission.Ensemble(models=[m_gram, m_gram_phrase, m_GRU, m_GRU_phrase], 
                               val_data=[val_data, val_data_phrase, val_data, val_data_phrase],
                               test_data=[test_data, test_data_phrase, test_data, val_data_phrase])
df_val = ensemble.get_ensemble_result(submission=False)

ensemble f1 val score: 0.8322322322322322


In [394]:
importlib.reload(submission)

ensemble = submission.Ensemble(models=[m_gram, m_gram_phrase, m_GRU, m_GRU_phrase], 
                               val_data=[val_data, val_data_phrase, val_data, val_data_phrase],
                               test_data=[test_data, test_data_phrase, test_data, val_data_phrase])
df_val = ensemble.get_ensemble_result(submission=False)

ensemble f1 val score: 0.8322322322322322


### Generate submission file for kaggle 

In [384]:
df = ensemble.get_ensemble_result(submission=True)
ensemble.save_submission_file("ft_pretrain_GRU")

/home/molly/Desktop/CSE244_ML_for_NLP/hw1/submission.py:87: RuntimeWarning: invalid value encountered in greater
  pred_idx = np.arange(num_class)[(preds[i, :] > 0.5).astype(bool)]


### Error analysis for best model

In [354]:
errors = df_val[df_val["CORE RELATIONS"]!=df_val.raw_label]; errors.shape
correct = df_val[df_val["CORE RELATIONS"]==df_val.raw_label]; errors.shape

(77, 3)

In [355]:
accuracy = 1-len(errors)/len(df_val); accuracy

0.7687687687687688

In [370]:
for category, count in df_val.raw_label.value_counts().iteritems():
    if count < 5:
        continue
    print(category)
    num_correct = correct[correct.raw_label.isin([category])].shape[0]
    accuracy = num_correct/count
    print("{0:.3f}".format(accuracy))

NO_REL
0.833
movie.directed_by
0.737
movie.starring.actor
0.862
movie.rating
0.870
movie_other
0.450
movie.subjects
1.000
other
0.474
movie.initial_release_date
1.000
movie.language
0.889
movie.produced_by
1.000
movie.estimated_budget
0.917
movie.country
0.909
movie.gross_revenue
1.000
movie.production_companies
0.889
movie.country movie.language
0.833
movie.genre
1.000


In [371]:
errors[errors.raw_label.isin(["other", "movie_other"])]

,CORE RELATIONS,text,raw_label
ID,,,
2844,other,find movie based on criteria,movie_other
2873,movie.genre,movie type,movie_other
2882,NO_REL,is kristen stewart really dating robert pattinson,movie_other
2889,NO_REL,middle aged white actors,movie_other
2896,movie.genre,search for dramas like downton abbey,movie_other
2914,movie.subjects,after watching this movie will i have nightmar...,movie_other
2922,movie_other,what do the actors speak,other
2939,movie.genre,scary,other
2943,movie.subjects other,zombies,other


In [372]:
correct[correct.raw_label.isin(["other", "movie_other"])]

,CORE RELATIONS,text,raw_label
ID,,,
2851,movie_other,search by box-office rating,movie_other
2855,movie_other,search by stars,movie_other
2857,movie_other,searching for cheap movies,movie_other
2861,movie_other,where can i rent this movie,movie_other
2894,movie_other,movies like in july,movie_other
2904,movie_other,what other movies are about the holocaust simi...,movie_other
2954,other,can i have a map of britain please,other
2972,other,find map of canada,other
2973,other,find select info,other


In [350]:
df_train = pd.read_csv("./data/train_real.csv", index_col="ID")[["text", "raw_label"]]
df_test = pd.read_csv("./data/holdout_test.csv", index_col="ID")[["text", "raw_label"]]
df_val = pd.read_csv("./data/val.csv", index_col="ID")[["text", "raw_label"]]
df_all_train = pd.read_csv("./data/train.csv", index_col="ID")[["text", "raw_label"]]

In [351]:
df_train[df_train.raw_label.isin(["NO_REL", "other"])]

,text,raw_label
ID,,
735,i need movie information for life is beautiful,NO_REL
737,i want to know more about the movie life is be...,NO_REL
738,i would like info on the movie life is beautiful,NO_REL
739,please show me info on the movie life is beaut...,NO_REL
740,search for the movie life is beautiful,NO_REL
742,can i see more info the movie life is beautiful,NO_REL
743,can you show me info on the movie life is beau...,NO_REL
744,display information for the movie life is beau...,NO_REL
773,i am looking for the information on the movie ...,NO_REL


In [265]:
df_val.index.value_counts().shape

(315,)

In [266]:
df_train.index.value_counts().shape

(2702,)

In [268]:
df_all_train.index.value_counts().shape

(3338,)

In [269]:
321 + 315 + 2702

3338

In [271]:
df_val.shape

(333, 2)